# Das Springerproblem

Beim [Springerproblem](https://de.wikipedia.org/wiki/Springerproblem) geht es darum, mit dem Springer auf dem Schachbrett nacheinander aller Felder zu besuchen, ohne eines doppelt zu betreten.

Dabei handelt es sich um eines der ältesten kombinatorischen Probleme der Welt, das mittlerweile auch theoretisch sehr gut untersucht ist.

Wir werden es in diesem Beispiel mithilfe von Python zu lösen versuchen.

## Backtracking-Algorithmus

Der einfachste Ansatz ist, jeweils irgendeinen Zug zu machen und bei Bedarf – wenn man in einer Sackgasse gelandet ist – Züge zurückzunehmen. 

In [ ]:
# Wir betrachten der Einfachheit halber nur quadratische Bretter
SIZE = 6


### Generatorfunktion für mögliche Züge

Wir stellen Koordinaten auf dem Brett als Tupel `(x, y)` dar und definieren eine Generatorfunktion `knight_moves(p)`, die alle möglichen Züge eines Springers liefert.

In [ ]:
def knight_moves(p):
    """ Generiere alle möglichen Züge eines Springers auf p 
    
    >>> { p for p in knight_moves( (0, 0) ) }
    {(1, 2), (2, 1)}
    
    >>> { p for p in knight_moves( (2, 2) ) }
    {(0, 1), (4, 1), (3, 0), (1, 4), (4, 3), (1, 0), (3, 4), (0, 3)}
    """
    (x, y) = p
    for (dx, dy) in [ (1, 2), (2, 1) ]:
        for (mx, my) in [ (x, y) for x in [ -1, 1 ] for y in [ -1, 1 ] ]:
            if x + dx*mx < 0 or x + dx*mx >= SIZE: continue
            if y + dy*my < 0 or y + dy*my >= SIZE: continue
            yield (x + dx*mx, y + dy*my)

In [ ]:
import doctest
doctest.testmod(verbose=True)

## Timing via Decorator

Wir wollen die Zeit messen, die der Algorithmus (in zwei verschiedenen Varianten) benötigt. Dazu definieren wir einen passenden Decorator `@timeit` 

In [29]:
import time

def timeit(method):
    """ Messe die Ausführungszeit der Methode method """
    
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print(f"execution time for {method.__name__}: {te - ts :.3f}")
        return result
    return timed

## Backtracking mit Rekursion

Für das Backtracking definieren wir eine rekursive Suchfunktion.

In [42]:
import random 

backtracks = 0
best = SIZE * SIZE

def search(p, visited, depth=0):
    """ Suche einen Pfad ab Punkt p, der die verbleibenden nicht besuchten Felder abdeckt.
    
    Wenn ein Pfad gefunden wurde, gebe True und den Pfad zurück.
    """
    global backtracks
    global best
    
    visited[p[0] * SIZE + p[1]] = True
    
    if visited.count(False) == 0:
        return (True, [p])
    
    else:
        possible_moves = [ q for q in knight_moves(p)]
        for q in possible_moves:
            if visited[q[0] * SIZE + q[1]]: continue     # q wurde chon besucht!
            (res, path) = search(q, visited[:], depth+1) # Suche weiter mit Kopie von visited!
            if res:
                path.append(p)
                return (True, path)
            else:
                backtracks += 1
                remain = visited.count(False)
                if remain < best:
                    best = remain
                if backtracks % 1000000 == 0:
                    print(f"backtracks={backtracks:,d}, depth={depth}, best={best}")
        else:
            return (False, [])
        
@timeit
def timed_search(size=6):
    global SIZE
    global backtracks
    global best
    SIZE = size
    backtracks = 0
    best = SIZE * SIZE
    visited = [ False for i in range(SIZE * SIZE) ]
    p = (0, 0)
    return search(p, visited)

timed_search(4)

execution time for timed_search: 0.010


(False, [])

In [43]:
timed_search(6)

backtracks=1,000,000, depth=27, best=2
backtracks=2,000,000, depth=26, best=2
backtracks=3,000,000, depth=25, best=2
backtracks=4,000,000, depth=21, best=2
backtracks=5,000,000, depth=28, best=2
backtracks=6,000,000, depth=20, best=2
backtracks=7,000,000, depth=23, best=2
backtracks=8,000,000, depth=26, best=2
backtracks=9,000,000, depth=24, best=2
backtracks=10,000,000, depth=26, best=2
backtracks=11,000,000, depth=22, best=2
backtracks=12,000,000, depth=27, best=2
backtracks=13,000,000, depth=25, best=2
backtracks=14,000,000, depth=26, best=2
backtracks=15,000,000, depth=21, best=2
execution time for timed_search: 57.016


(True,
 [(5, 0),
  (4, 2),
  (3, 0),
  (5, 1),
  (3, 2),
  (2, 0),
  (0, 1),
  (2, 2),
  (4, 1),
  (5, 3),
  (4, 5),
  (2, 4),
  (0, 5),
  (1, 3),
  (2, 5),
  (4, 4),
  (5, 2),
  (4, 0),
  (2, 1),
  (3, 3),
  (5, 4),
  (3, 5),
  (1, 4),
  (0, 2),
  (1, 0),
  (3, 1),
  (4, 3),
  (5, 5),
  (3, 4),
  (1, 5),
  (0, 3),
  (1, 1),
  (2, 3),
  (0, 4),
  (1, 2),
  (0, 0)])

## Warnsdorf-Heuristik

Bereits im Jahr 1823 (!) schlug H. C. von Warnsdorf eine Heuristik vor, die das Problem vereinfacht: 

> In jedem Zug wählt der Springer das Feld aus, von dem aus er die wenigsten weiteren unbesuchten Felder zur
> Auswahl hat.  

In [21]:
def warnsdorf(moves, visited):
    """ Sortiere moves nach der Warnsdorf-Heuristik 
    Sortiere die Züge aufsteigend nach der Anzahl verbleibender Felder vom Zielfeld aus
    """
    counts = {}
    for q in moves:
        count = 0
        for s in knight_moves(q):
            if not visited[s[0] * SIZE + s[1]]:
                count += 1
        counts[q] = count
    
    return sorted(moves, key = lambda p : counts[p])
    
visited = [ False for i in range(SIZE * SIZE) ]
moves = [ p for p in knight_moves( (1, 2) ) ]
warnsdorf(moves, visited)

[(0, 0), (0, 4), (2, 0), (2, 4), (3, 1), (3, 3)]

In [47]:
import random 

backtracks = 0
best = SIZE * SIZE


def search(p, visited, depth=0):
    """ Suche einen Pfad ab Punkt p, der die verbleibenden nicht besuchten Felder abdeckt.
    
    Wenn ein Pfad gefunden wurde, gebe True und den Pfad zurück.
    """
    global backtracks
    global best
    
    visited[p[0] * SIZE + p[1]] = True
    
    if visited.count(False) == 0:
        return (True, [p])
    
    else:
        possible_moves = warnsdorf([ q for q in knight_moves(p)], visited)
        for q in possible_moves:
            if visited[q[0] * SIZE + q[1]]: continue     # q wurde chon besucht!
            (res, path) = search(q, visited[:], depth+1) # Suche weiter mit Kopie von visited!
            if res:
                path.append(p)
                return (True, path)
            else:
                backtracks += 1
                remain = visited.count(False)
                if remain < best:
                    best = remain
                if backtracks % 1000000 == 0:
                    print(f"backtracks={backtracks:,d}, depth={depth}, left={best}")
        else:
            return (False, [])
        


        
@timeit
def timed_search(size=6):
    global SIZE
    SIZE = size
    visited = [ False for i in range(SIZE * SIZE) ]
    p = (0, 0)
    return search(p, visited)

timed_search(6)

execution time for timed_search: 0.001


(True,
 [(4, 3),
  (5, 5),
  (3, 4),
  (2, 2),
  (1, 4),
  (3, 5),
  (2, 3),
  (1, 5),
  (0, 3),
  (1, 1),
  (3, 0),
  (5, 1),
  (3, 2),
  (2, 4),
  (0, 5),
  (1, 3),
  (0, 1),
  (2, 0),
  (4, 1),
  (5, 3),
  (4, 5),
  (3, 3),
  (5, 4),
  (4, 2),
  (5, 0),
  (3, 1),
  (1, 0),
  (0, 2),
  (2, 1),
  (4, 0),
  (5, 2),
  (4, 4),
  (2, 5),
  (0, 4),
  (1, 2),
  (0, 0)])

In [48]:
timed_search(8)

execution time for timed_search: 0.006


(True,
 [(4, 3),
  (3, 1),
  (2, 3),
  (3, 5),
  (1, 4),
  (0, 2),
  (1, 0),
  (2, 2),
  (3, 4),
  (4, 2),
  (2, 1),
  (3, 3),
  (5, 4),
  (7, 3),
  (6, 1),
  (4, 0),
  (5, 2),
  (4, 4),
  (3, 2),
  (5, 3),
  (4, 5),
  (2, 4),
  (3, 6),
  (5, 5),
  (7, 4),
  (6, 6),
  (4, 7),
  (2, 6),
  (0, 7),
  (1, 5),
  (0, 3),
  (1, 1),
  (3, 0),
  (5, 1),
  (7, 0),
  (6, 2),
  (5, 0),
  (7, 1),
  (6, 3),
  (7, 5),
  (6, 7),
  (4, 6),
  (2, 7),
  (0, 6),
  (2, 5),
  (1, 7),
  (0, 5),
  (1, 3),
  (0, 1),
  (2, 0),
  (4, 1),
  (6, 0),
  (7, 2),
  (6, 4),
  (7, 6),
  (5, 7),
  (6, 5),
  (7, 7),
  (5, 6),
  (3, 7),
  (1, 6),
  (0, 4),
  (1, 2),
  (0, 0)])